# Example how to query multiple items using paging

In [1]:
from sgqlc.operation import Operation
from pdp_schema import pdp_schema as schema
from pdp_graphql_client_python import client

In [2]:
! echo "${PDP_API}"

https://graphql-api.pdp.int.srgssr.ch/graphql


In [3]:
# show available queries
# *Page queries allow for paging (https://graphql.org/learn/pagination/)
# first=[number of queries to retrieve, e.g. *first* 10 queries]
# after=[id or cursor position from which to start]
schema.Query

type Query {
  faroProgramPage(first: Int!, after: String): FaroProgramPage!
  faroPrograms(ids: [String!]!): [FaroProgram]!
  randomFaroPrograms: [FaroProgram!]!
  faroItemPage(first: Int!, after: String): FaroItemPage!
  faroItems(ids: [String!]!): [FaroItem]!
  faroItemsByPlayUrn(urns: [String!]!): [FaroItem]!
  faroItemsByPlayUrnGraphDb(urns: [String!]!): [FaroItem]!
  randomFaroItems: [FaroItem!]!
}

In [4]:
# retrieve query operations
op = Operation(schema.Query)

In [5]:
type(op)

sgqlc.operation.Operation

In [6]:
# return current query (empty)
op

query {
}

In [7]:
# select query type and configure with argument
faro = op.faro_program_page(first=10)

In [8]:
# return current query (empty)
op

query {
  faroProgramPage(first: 10) {
    edges {
      id
      bu
      mediaType
      episodeIds
      sTit
      sDatStart
      sGef
      pTit
      department
      workgroup
      seriesNr
    }
    cursor
  }
}

In [9]:
# select fields to be retured
faro.edges.media_type()
faro.edges.s_dat_start()
faro.edges.s_gef()
faro.edges.s_tit()

sTit

In [10]:
# return current query
op

query {
  faroProgramPage(first: 10) {
    edges {
      mediaType
      sDatStart
      sGef
      sTit
    }
  }
}

In [11]:
# run query and retrieve data
data = client.run_query(op)

In [12]:
# display raw data (json)
data

{'data': {'faroProgramPage': {'edges': [{'mediaType': 'audio',
     'sDatStart': '1994-06-26',
     'sGef': 'Doppelpunkt',
     'sTit': None},
    {'mediaType': 'audio',
     'sDatStart': '2012-11-26',
     'sGef': None,
     'sTit': None},
    {'mediaType': 'audio',
     'sDatStart': '2006-11-21',
     'sGef': 'Oz',
     'sTit': 'Contribuziun'},
    {'mediaType': 'audio',
     'sDatStart': '2012-09-28',
     'sGef': None,
     'sTit': None},
    {'mediaType': None, 'sDatStart': None, 'sGef': None, 'sTit': None},
    {'mediaType': 'audio',
     'sDatStart': '2018-04-21',
     'sGef': None,
     'sTit': None},
    {'mediaType': 'audio',
     'sDatStart': '1979-03-20',
     'sGef': 'Wir lesen vor',
     'sTit': 'Fortsetzung folgt: Eine Krähe war mit mir\n'},
    {'mediaType': 'audio',
     'sDatStart': '2009-06-18',
     'sGef': None,
     'sTit': None},
    {'mediaType': 'audio',
     'sDatStart': '2005-04-20',
     'sGef': 'Kulturtipp',
     'sTit': None},
    {'mediaType': 'audio',
  

In [13]:
# convert again to object oriented results
results = (op + data).faro_program_page

In [14]:
# display object transformed data
results

FaroProgramPage(edges=[FaroProgram(media_type='audio', s_dat_start='1994-06-26', s_gef='Doppelpunkt', s_tit=None), FaroProgram(media_type='audio', s_dat_start='2012-11-26', s_gef=None, s_tit=None), FaroProgram(media_type='audio', s_dat_start='2006-11-21', s_gef='Oz', s_tit='Contribuziun'), FaroProgram(media_type='audio', s_dat_start='2012-09-28', s_gef=None, s_tit=None), FaroProgram(media_type=None, s_dat_start=None, s_gef=None, s_tit=None), FaroProgram(media_type='audio', s_dat_start='2018-04-21', s_gef=None, s_tit=None), FaroProgram(media_type='audio', s_dat_start='1979-03-20', s_gef='Wir lesen vor', s_tit='Fortsetzung folgt: Eine Krähe war mit mir\n'), FaroProgram(media_type='audio', s_dat_start='2009-06-18', s_gef=None, s_tit=None), FaroProgram(media_type='audio', s_dat_start='2005-04-20', s_gef='Kulturtipp', s_tit=None), FaroProgram(media_type='audio', s_dat_start='2002-01-28', s_gef='Regionaljournal Zürich Schaffhausen', s_tit=None)])

In [15]:
# discover data types
type(results.edges[0])

type FaroProgram {
  id: String!
  bu: String
  mediaType: String
  episodeIds: [String!]!
  sTit: String
  sDatStart: String
  sGef: String
  pTit: String
  department: String
  workgroup: String
  seriesNr: Int
  moderators: [FaroModerator!]!
  itemPage(first: Int!, after: String): FaroItemPage!
  playEpisodes: [PlayEpisode!]!
}

In [16]:
from collections import Counter
from datetime import date

# count year occurrences
Counter(date.fromisoformat(x.s_dat_start).year for x in results.edges if x.s_dat_start)

Counter({1994: 1,
         2012: 2,
         2006: 1,
         2018: 1,
         1979: 1,
         2009: 1,
         2005: 1,
         2002: 1})

In [17]:
import pandas as pd

In [18]:
dir(results.edges[0])

['_ContainerTypeMeta__fields',
 '_ContainerType__get_type_for_selection',
 '_ContainerType__populate_field_data',
 '_ContainerType__populate_fields',
 '_ContainerType__populate_fields_from_selection_list',
 '__bytes__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__field_names__',
 '__fields_cache__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__interfaces__',
 '__iter__',
 '__json_data__',
 '__json_dump_args__',
 '__kind__',
 '__le__',
 '__len__',
 '__lt__',
 '__meta_fields__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__schema__',
 '__selection_list__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__to_json_value__',
 '__weakref__',
 'media_type',
 's_dat_start',
 's_gef',
 's_tit']

In [19]:
results.edges[0].__field_names__

('id',
 'bu',
 'media_type',
 'episode_ids',
 's_tit',
 's_dat_start',
 's_gef',
 'p_tit',
 'department',
 'workgroup',
 'series_nr',
 'moderators',
 'item_page',
 'play_episodes')

In [20]:
{key: value for key, value in results.edges[0].__dict__.items() if not key.startswith("__")}

{'media_type': 'audio',
 's_dat_start': '1994-06-26',
 's_gef': 'Doppelpunkt',
 's_tit': None}

In [21]:
df = pd.DataFrame(data['data']['faroProgramPage']['edges'])

In [22]:
df

,mediaType,sDatStart,sGef,sTit
0,audio,1994-06-26,Doppelpunkt,None
1,audio,2012-11-26,None,None
2,audio,2006-11-21,Oz,Contribuziun
3,audio,2012-09-28,None,None
4,None,None,None,None
5,audio,2018-04-21,None,None
6,audio,1979-03-20,Wir lesen vor,Fortsetzung folgt: Eine Krähe war mit mir\n
7,audio,2009-06-18,None,None
8,audio,2005-04-20,Kulturtipp,None
9,audio,2002-01-28,Regionaljournal Zürich Schaffhausen,None
